In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/12 16:15:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1

    Read the case, department, and source data into their own spark dataframes.

    Let's see how writing to the local disk works in spark:
        Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
        Inspect your folder structure. What do you notice?

    Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.


df = spark.read.csv("data/source.csv", sep=",", header=True, inferSchema=True)

OR

(
    spark.read.format("csv")
    .option("sep", ",")
    .option("inferSchema", True)
    .option("header", True)
    .load("data/source.csv")
)


In [2]:
# Read

In [3]:
case = spark.read.csv('case.csv', sep=",", header=True, inferSchema=True)

source = spark.read.csv('source.csv', sep=",", header=True, inferSchema=True)

dept = spark.read.csv('dept.csv', sep=",", header=True, inferSchema=True)




In [4]:
case.show(5, vertical = True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11          
 SLA_due_date         | 1/5/18 8:30          
 case_late            | NO                   
 num_days_late        | -2.0126041

In [5]:
source.show(5)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
|   119403|  Betty De Hoyos|
|   119555|  Socorro Quiara|
+---------+----------------+
only showing top 5 rows



In [6]:
dept.show(5, vertical = True)

-RECORD 0--------------------------------------
 dept_division          | 311 Call Center      
 dept_name              | Customer Service     
 standardized_dept_name | Customer Service     
 dept_subject_to_SLA    | YES                  
-RECORD 1--------------------------------------
 dept_division          | Brush                
 dept_name              | Solid Waste Manag... 
 standardized_dept_name | Solid Waste          
 dept_subject_to_SLA    | YES                  
-RECORD 2--------------------------------------
 dept_division          | Clean and Green      
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 3--------------------------------------
 dept_division          | Clean and Green N... 
 dept_name              | Parks and Recreation 
 standardized_dept_name | Parks & Recreation   
 dept_subject_to_SLA    | YES                  
-RECORD 4-------------------------------

from pyspark.sql.types import StructType, StructField, StringType

schema = StructType(
    [
        StructField("dept_division", StringType()),
        StructField("dept_name", StringType()),
        StructField("standardized_dept_name", StringType()),
        StructField("dept_subject_to_SLA", StringType())
    ]
)

spark.read.csv("dept.csv", header=True, schema=schema)


dept_division,dept_name,standardized_dept_name,dept_subject_to_SLA

# Write

mpg.write.json("data/mpg_json", mode="overwrite")

- like much else in spark, there's multiple ways we could do this:
(
    mpg.write.format("csv")
    .mode("overwrite")
    .option("header", "true")
    .save("data/mpg_csv")
)

In [7]:
source.write.json('source_json', mode = 'overwrite')

In [8]:
case.write.json('case_json', mode = 'overwrite')

In [9]:
dept.write.json('dept_json', mode = 'overwrite')

In [10]:
source.write.csv('source_csv', mode = 'overwrite')

In [11]:
case.write.csv('case_csv', mode = 'overwrite')

In [12]:
dept.write.csv('dept_csv', mode = 'overwrite')

In [ ]:
# observation: They appeared as folders and were imported in parts

In [13]:
case.printSchema() #dates are still strings, case_late and case_closed are boolean columns

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [14]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [15]:
dept.printSchema() #might wanna make yes/no column a 0/1 column

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [16]:
# above are my notes, what I would do. The first few lines below are what the curriculum directs me to do. 

In [17]:
#c
case = case.withColumnRenamed("SLA_due_date", "case_due_date")

In [19]:
#c
# demonstrating we only have yes/no in each field
case.groupBy("case_closed", "case_late").count().show()


+-----------+---------+------+
|case_closed|case_late| count|
+-----------+---------+------+
|         NO|      YES|  6525|
|        YES|      YES| 87978|
|         NO|       NO| 11585|
|        YES|       NO|735616|
+-----------+---------+------+



In [20]:
#c
case = case.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

case.select("case_closed", "case_late").show(5)


+-----------+---------+
|case_closed|case_late|
+-----------+---------+
|       true|    false|
|       true|    false|
|       true|    false|
|       true|    false|
|       true|     true|
+-----------+---------+
only showing top 5 rows



In [21]:
# c
# The council_district column appears as though it is an integer, 
#but this is just a unique identifier for each district
case = case.withColumn("council_district", col("council_district").cast("string"))


In [22]:
# c
print("--- Before handling dates")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)

fmt = "M/d/yy H:mm"
case = (
    case.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

print("--- After")
case.select("case_opened_date", "case_closed_date", "case_due_date").show(5)


--- Before handling dates
+----------------+----------------+-------------+
|case_opened_date|case_closed_date|case_due_date|
+----------------+----------------+-------------+
|     1/1/18 0:42|    1/1/18 12:29| 9/26/20 0:42|
|     1/1/18 0:46|     1/3/18 8:11|  1/5/18 8:30|
|     1/1/18 0:48|     1/2/18 7:57|  1/5/18 8:30|
|     1/1/18 1:29|     1/2/18 8:13| 1/17/18 8:30|
|     1/1/18 1:34|    1/1/18 13:29|  1/1/18 4:34|
+----------------+----------------+-------------+
only showing top 5 rows

--- After
+-------------------+-------------------+-------------------+
|   case_opened_date|   case_closed_date|      case_due_date|
+-------------------+-------------------+-------------------+
|2018-01-01 00:42:00|2018-01-01 00:42:00|2018-01-01 00:42:00|
|2018-01-01 00:46:00|2018-01-01 00:46:00|2018-01-01 00:46:00|
|2018-01-01 00:48:00|2018-01-01 00:48:00|2018-01-01 00:48:00|
|2018-01-01 01:29:00|2018-01-01 01:29:00|2018-01-01 01:29:00|
|2018-01-01 01:34:00|2018-01-01 01:34:00|2018-01-01 01:

In [25]:
# c
# Using the trim and lower functions lets us strip any leading or trailing whitespace 
#and convert everything to lowercase.
print("--- Before")
case.select("request_address").show(5)

case = case.withColumn("request_address", trim(lower(case.request_address)))

print("--- After")
case.select("request_address").show(5)


--- Before
+--------------------+
|     request_address|
+--------------------+
|2315  EL PASO ST,...|
|2215  GOLIAD RD, ...|
|102  PALFREY ST W...|
|114  LA GARDE ST,...|
|734  CLEARVIEW DR...|
+--------------------+
only showing top 5 rows

--- After
+--------------------+
|     request_address|
+--------------------+
|2315  el paso st,...|
|2215  goliad rd, ...|
|102  palfrey st w...|
|114  la garde st,...|
|734  clearview dr...|
+--------------------+
only showing top 5 rows



# notes
how to add leading zeros to concil columm

df = df.withColumn("council_district", col("council_district").cast("int"))

 '%03d' means at least 3 digits, pad with 0s

 In order to use the format_string function the way we are, we'll need to
 convert council_district back to an integer temporarily, but the final output
 will be a string.

df = df.withColumn(
    "council_district",
    format_string("%03d", col("council_district").cast("int")),
)

df.select("council_district").show(5)


In [26]:
# c
# set zipcode column
case = case.withColumn("zipcode", regexp_extract("request_address", r"\d+$", 0))

case.select("zipcode").show(5)



+-------+
|zipcode|
+-------+
|  78207|
|  78223|
|  78223|
|  78223|
|  78228|
+-------+
only showing top 5 rows



In [ ]:
#  Join w/ dept dataset

In [29]:
# c
comb = (
    case
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .drop(case.dept_division)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

comb.show(2, vertical=True)


-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 case_due_date        | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 zipcode              | 78207                
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 2018-01-01 00:46:00  
 case_closed_date     | 2018-01-01 00:46:00  
 case_due_date        | 2018-01-01

In [ ]:
# train test split

train, test = comb.randomSplit([0.8, 0.2])
#OR
train, validate, test = comb.randomSplit([0.6, 0.2, 0.2])



    How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?
    How many Stray Animal cases are there?

    How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

    Convert the council_district column to a string column.
    Extract the year from the case_closed_date column.

    Convert num_days_late from days to hours in new columns num_hours_late.

    Join the case data with the source and department data.

    Are there any cases that do not have a request source?

    What are the top 10 service request types in terms of number of requests?
    What are the top 10 service request types in terms of average days late?
    Does number of days late depend on department?
    How do number of days late depend on department and request type?

You might have noticed that the latest date in the dataset is fairly far off from the present day. To account for this, replace any occurances of the current time with the maximum date from the dataset.